In [ ]:
import numpy as np
import pickle
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
working_dir='../input/'

In [ ]:
test_batch = unpickle(working_dir+'test_batch')
meta_batch=unpickle(working_dir+'batches.meta')

In [ ]:
test_batch.keys()

In [ ]:
meta_batch.keys()

In [ ]:
Counter(test_batch[b'labels'])

In [ ]:
test_batch[b'data'].shape

In [ ]:
data_r = test_batch[b'data'][:, :1024].reshape(-1,32, 32,)
data_g = test_batch[b'data'][:, 1024:2*1024].reshape(-1, 32, 32)
data_b = test_batch[b'data'][:, 2*1024:].reshape(-1, 32, 32)

In [ ]:
test_batch[b'data'].shape


In [ ]:
viewable_images = np.stack([data_r, data_g, data_b], axis=-1)
viewable_images.shape


In [ ]:
for i in range(0,10):
    plt.subplot(2,5,1+i)
    plt.imshow(viewable_images[i])

In [ ]:
labels = np.array(test_batch[b'labels'])
labels.shape


In [ ]:
scaler=MinMaxScaler()
scaled_test_data=scaler.fit_transform(test_batch[b'data'])

In [ ]:
data_r = scaled_test_data[:, :1024].reshape(-1,32, 32,)
data_g = scaled_test_data[:, 1024:2*1024].reshape(-1, 32, 32)
data_b = scaled_test_data[:, 2*1024:].reshape(-1, 32, 32)

In [ ]:
viewable1= np.stack([data_r, data_g, data_b], axis=-1)

In [ ]:
viewable1.shape

In [ ]:
for i in range(0,10):
    plt.subplot(2,5,1+i)
    plt.imshow(viewable1[i])

In [ ]:
test_batch[b'data']

In [ ]:
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(32, 32, 3), activation='relu', bias=True))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
 

In [ ]:
 train_image, test_image, train_labels, test_labels = train_test_split(viewable1, labels)

In [ ]:
train_image.shape

In [ ]:
model.fit(train_image, train_labels, validation_data=(test_image, test_labels),epochs=20)

In [ ]:
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
def show_image(number):
    fig = plt.figure(figsize = (15,8))
    test_image1 = np.expand_dims(test_image[number], axis=0)
    test_result = model.predict_classes(test_image1)
    plt.imshow(test_image[number])
    dict_key = test_result[0]
    plt.title("Predicted: {}, Actual: {}".format(labels[dict_key], labels[test_labels[number]]))


In [ ]:
show_image(17)